In [2]:
import pandas as pd
import numpy as np
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, LSTM, Conv1D, MaxPooling1D,Bidirectional


In [4]:
df = pd.read_csv('/content/drive/MyDrive/stock_train.csv')

In [6]:
train_df = df[df['종목코드'] == 'A060310' ].drop(['종목명' , '종목코드'], axis=1).reset_index()
train_df

,index,일자,거래량,시가,고가,저가,종가
0,0,20210601,166690,2890,2970,2885,2920
1,2000,20210602,134833,2915,2975,2830,2900
2,4000,20210603,144470,2900,2925,2875,2900
3,6000,20210604,934224,2930,3120,2920,2950
4,8000,20210607,946560,3000,3150,2955,3150
...,...,...,...,...,...,...,...
489,978000,20230523,117084,2300,2315,2260,2285
490,980000,20230524,67101,2280,2300,2255,2280
491,982000,20230525,91789,2260,2285,2235,2245
492,984000,20230526,13590249,2245,2915,2230,2910


In [7]:
def MinMaxScaler(data):
    numerator = data - np.min(data, 0)
    denominator = np.max(data, 0) - np.min(data, 0)

    return numerator / (denominator + 1e-7)

In [8]:
train_df.columns

Index(['index', '일자', '거래량', '시가', '고가', '저가', '종가'], dtype='object')

In [9]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   index   494 non-null    int64
 1   일자      494 non-null    int64
 2   거래량     494 non-null    int64
 3   시가      494 non-null    int64
 4   고가      494 non-null    int64
 5   저가      494 non-null    int64
 6   종가      494 non-null    int64
dtypes: int64(7)
memory usage: 27.1 KB


In [10]:
dfx = train_df[[ '일자',  '거래량', '시가', '고가', '저가', '종가']]
dfx = MinMaxScaler(dfx)

dfy = dfx[['종가']]
dfx = dfx[['일자',  '거래량', '시가', '고가', '저가']]

dfx

,일자,거래량,시가,고가,저가
0,0.000000,0.001621,0.362534,0.347349,0.420705
1,0.000050,0.001207,0.372311,0.349177,0.396476
2,0.000100,0.001332,0.366445,0.330896,0.416300
3,0.000151,0.011601,0.378178,0.402194,0.436123
4,0.000301,0.011761,0.405553,0.413163,0.451542
...,...,...,...,...,...
489,0.999649,0.000976,0.131795,0.107861,0.145374
490,0.999699,0.000326,0.123973,0.102377,0.143172
491,0.999749,0.000647,0.116152,0.096892,0.134361
492,0.999799,0.176155,0.110285,0.327239,0.132159


In [11]:
time_step = 10
step = 1

def create_dataset(X, y, time_step, step):
    Xs, ys = [], []

    for i in range(0, len(X) - time_step, step):
        v = X.iloc[i : (i + time_step)].values
        labels = y.iloc[i + time_step]

        Xs.append(v)
        ys.append(labels)

    return np.array(Xs), np.array(ys)

In [12]:
X, y = create_dataset(dfx, dfy, time_step, step)

X.shape

(484, 10, 5)

In [13]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, shuffle=False)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((324, 10, 5), (160, 10, 5), (324, 1), (160, 1))

In [14]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, Dropout

# RNN

In [38]:
model = Sequential([
    SimpleRNN(20, activation='relu', input_shape=(X.shape[1:]), return_sequences=True),
    Dropout(0.5),
    SimpleRNN(20, activation='relu'),
    Dropout(0.3),
    Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn (SimpleRNN)      (None, 10, 20)            520       
                                                                 
 dropout (Dropout)           (None, 10, 20)            0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 20)                820       
                                                                 
 dropout_1 (Dropout)         (None, 20)                0         
                                                                 
 dense (Dense)               (None, 1)                 21        
                                                                 
Total params: 1361 (5.32 KB)
Trainable params: 1361 (5.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.33, shuffle=False)


Epoch 1/500
7/7 [==============================] - 4s 75ms/step - loss: 0.3604 - val_loss: 0.0384
Epoch 2/500
7/7 [==============================] - 0s 15ms/step - loss: 0.2596 - val_loss: 0.0189
Epoch 3/500
7/7 [==============================] - 0s 14ms/step - loss: 0.2304 - val_loss: 0.0153
Epoch 4/500
7/7 [==============================] - 0s 13ms/step - loss: 0.1523 - val_loss: 0.0166
Epoch 5/500
7/7 [==============================] - 0s 13ms/step - loss: 0.1249 - val_loss: 0.0182
Epoch 6/500
7/7 [==============================] - 0s 14ms/step - loss: 0.0917 - val_loss: 0.0178
Epoch 7/500
7/7 [==============================] - 0s 13ms/step - loss: 0.0992 - val_loss: 0.0136
Epoch 8/500
7/7 [==============================] - 0s 14ms/step - loss: 0.0698 - val_loss: 0.0118
Epoch 9/500
7/7 [==============================] - 0s 12ms/step - loss: 0.0656 - val_loss: 0.0090
Epoch 10/500
7/7 [==============================] - 0s 12ms/step - loss: 0.0725 - val_loss: 0.0067
Epoch 11/500
7/7 [=

In [40]:
from sklearn.metrics import mean_squared_error as mse

y_pred = model.predict(X_test)
print(mse(y_test, y_pred))

5/5 [==============================] - 0s 4ms/step
0.029429487749666843


# LSTM

In [50]:
model = Sequential([
    LSTM(20, activation='relu', input_shape=(X.shape[1:]), return_sequences=True),
    Dropout(0.5),
    LSTM(20, activation='relu'),
    Dropout(0.3),
    Dense(1)
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 20)            2080      
                                                                 
 dropout_4 (Dropout)         (None, 10, 20)            0         
                                                                 
 lstm_1 (LSTM)               (None, 20)                3280      
                                                                 
 dropout_5 (Dropout)         (None, 20)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 21        
                                                                 
Total params: 5381 (21.02 KB)
Trainable params: 5381 (21.02 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [51]:
model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, y_train, epochs=500, batch_size=32, validation_split=0.33, shuffle=False)


Epoch 1/500
7/7 [==============================] - 4s 78ms/step - loss: 0.3822 - val_loss: 0.1279
Epoch 2/500
7/7 [==============================] - 0s 19ms/step - loss: 0.2923 - val_loss: 0.0819
Epoch 3/500
7/7 [==============================] - 0s 17ms/step - loss: 0.2124 - val_loss: 0.0438
Epoch 4/500
7/7 [==============================] - 0s 19ms/step - loss: 0.1102 - val_loss: 0.0126
Epoch 5/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0675 - val_loss: 0.0056
Epoch 6/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0765 - val_loss: 0.0103
Epoch 7/500
7/7 [==============================] - 0s 17ms/step - loss: 0.0623 - val_loss: 0.0050
Epoch 8/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0603 - val_loss: 0.0047
Epoch 9/500
7/7 [==============================] - 0s 19ms/step - loss: 0.0520 - val_loss: 0.0054
Epoch 10/500
7/7 [==============================] - 0s 18ms/step - loss: 0.0442 - val_loss: 0.0077
Epoch 11/500
7/7 [=

In [53]:
from sklearn.metrics import mean_squared_error as mse

y_pred = model.predict(X_test)
print(mse(y_test, y_pred))

5/5 [==============================] - 0s 7ms/step
0.06123341436115496


# CNN

In [15]:
model = Sequential([
    Conv1D(64, 2 , activation='relu', input_shape=(X.shape[1:])),
    Dropout(0.5),
    Conv1D(64, 5 , activation='relu'),
    Flatten(),
    Dense(10 , activation = 'relu'),
    Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 9, 64)             704       
                                                                 
 dropout (Dropout)           (None, 9, 64)             0         
                                                                 
 conv1d_1 (Conv1D)           (None, 5, 64)             20544     
                                                                 
 flatten (Flatten)           (None, 320)               0         
                                                                 
 dense (Dense)               (None, 10)                3210      
                                                                 
 dense_1 (Dense)             (None, 1)                 11        
                                                                 
Total params: 24469 (95.58 KB)
Trainable params: 24469 (

In [16]:
model.compile(loss='mse', optimizer='adam')
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.33, shuffle=False)


Epoch 1/100
7/7 [==============================] - 2s 57ms/step - loss: 0.0982 - val_loss: 0.0200
Epoch 2/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0517 - val_loss: 0.0273
Epoch 3/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0373 - val_loss: 0.0111
Epoch 4/100
7/7 [==============================] - 0s 16ms/step - loss: 0.0176 - val_loss: 0.0069
Epoch 5/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0189 - val_loss: 0.0046
Epoch 6/100
7/7 [==============================] - 0s 15ms/step - loss: 0.0138 - val_loss: 0.0059
Epoch 7/100
7/7 [==============================] - 0s 12ms/step - loss: 0.0125 - val_loss: 0.0045
Epoch 8/100
7/7 [==============================] - 0s 13ms/step - loss: 0.0125 - val_loss: 0.0045
Epoch 9/100
7/7 [==============================] - 0s 15ms/step - loss: 0.0111 - val_loss: 0.0045
Epoch 10/100
7/7 [==============================] - 0s 14ms/step - loss: 0.0096 - val_loss: 0.0044
Epoch 11/100
7/7 [=

In [17]:
from sklearn.metrics import mean_squared_error as mse

y_pred = model.predict(X_test)
print(mse(y_test, y_pred))

5/5 [==============================] - 0s 3ms/step
0.010264695621589303
